## I solemnly swear that I have not discussed my assignment solutions with anyone in any way and the solutions I am submitting are my own personal work.
## Full Name: Karan Singh

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import io
import requests

### Loading our Dataset

In [2]:
pd.set_option('display.max_columns', None) 

df1 = pd.read_csv('A2_Q2.csv')
df=df1.copy()

print(df.shape)
df.head()

(30, 3)


,ID,Target,Score
0,1,False,0.46
1,2,False,0.14
2,3,False,0.48
3,4,True,0.91
4,5,False,0.24


## Part A Solution

In this part we are creating a new column prediction with our given threshold value 0.5.

In [3]:
df['Prediction'] = np.where(df['Score']>=0.5, 'True', 'False') 

In [4]:
df.head()

,ID,Target,Score,Prediction
0,1,False,0.46,False
1,2,False,0.14,False
2,3,False,0.48,False
3,4,True,0.91,True
4,5,False,0.24,False


To create the Confusion Matrix using pandas, we’ll need to apply the pd.crosstab as follows:

In [5]:
df = pd.DataFrame(df, columns=['Target','Prediction'])
confusion_matrix = pd.crosstab(df['Target'], df['Prediction'], rownames=['Target'], colnames=['Prediction'], margins = True)

In [6]:
confusion_matrix

Prediction,False,True,All
Target,,,
False,11,6,17
True,4,9,13
All,15,15,30


In [7]:
TP = confusion_matrix.iloc[1,1]
FP = confusion_matrix.iloc[0,1]
TN = confusion_matrix.iloc[0,0]
FN = confusion_matrix.iloc[1,0]

A confusion matrix is a square matrix  M  constructed such that  Mi,j  is equal to the number of observations known to be in group  i  but predicted to be in group  j . For a binary classifier,

M0,0  = True negatives (TN)
M1,0  = False negative (FN)
M1,1  = True positives (TP)
M0,1  = False positives (FP)

So, we calculated our Confusion Matrix and assigned row and column names.

## Part B Solution

In [8]:
error_rate = (FP+FN)/(TP+TN+FP+FN) #Calculating Error Rate
print(error_rate)

0.3333333333333333


Precsion tells us about when it predicts yes, how often is it correct.

In [9]:
precision = (TP)/(TP+FP) #Calculating Precision
print(precision)

0.6


TPR gives us an idea about when it’s actually yes, how often does it predict yes.

In [10]:
TPR = (TP)/(TP+FN) #Calculating Total Positive Rate
print(TPR)

0.6923076923076923


We calculate an F-measure which uses Harmonic Mean in place of Arithmetic Mean as it punishes the extreme values more.

In [11]:
F1 = 2 * (precision*TPR)/(precision+TPR) #Calculating F-1 score
print(F1)

0.6428571428571429


In [12]:
TNR = (TN)/(TN+FP) #Calculating Total Negative Rate
FPR = 1-TNR
print(FPR)

0.3529411764705882


In [13]:
data = [['error_rate',round(error_rate, 3)],['precision',round(precision, 3)],['TPR',round(TPR, 3)],['F1',round(F1, 3)],['FPR',round(FPR, 3)]]
df_metrics = pd.DataFrame(data,columns=['Metric','Value'])
df_metrics

,Metric,Value
0,error_rate,0.333
1,precision,0.600
2,TPR,0.692
3,F1,0.643
4,FPR,0.353


After calculating all the required values we used them in our new data frame with Metric and Value Columns.

## Part C Solution

In [14]:
thres=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
thr=0
Threshold=[]
TPR=[]
FPR=[]
reqd_cols =['Score']
df2=df1.copy()
for i in range(9):
    df2=df1.copy()
    #inc =0.1
    #thr=thr+inc
    thr=thres[i]
    tn=0
    tp=0
    fn=0
    fp=0
    recall=0
    TNR=0
    falsepr=0
    
    #changing all values of Score >= to threshold to 1&0 otherwise
    df2.loc[(df2.Score < thr), 'Score']= 0
    df2.loc[(df2.Score >= thr), 'Score']= 1
    df2["Score"]=df2["Score"].astype(int)
    confusion_matrix=pd.crosstab(index=df2['Target'], columns=df2['Score'], colnames=['Prediction'],rownames=['Target']).rename(columns={0: "Negative", 1: "Positive"})
    
    #print(confusion_matrix)
    #accessing conf matrix values
    tn=confusion_matrix['Negative'][False]
    tp=confusion_matrix['Positive'][True]
    fn=confusion_matrix['Negative'][True]
    fn=confusion_matrix['Positive'][False]
    recall = (tp)/(tp+fn)
    TNR=(tn)/(tn+fp)
    falsepr=1 - TNR
    Threshold.append(thr)
    TPR.append(recall)
    FPR.append(recall)
    FPR.append(falsepr)

In [15]:
#print (Threshold)
#print (TPR)
#print (FPR)

#dict = {'Threshold': Threshold, 'TPR': TPR, 'FPR': FPR}

a = {'Threshold': Threshold, 'TPR': TPR, 'FPR': FPR}
df_roc = pd.DataFrame.from_dict(a, orient='index')
df_roc.transpose().head()

,Threshold,TPR,FPR
0,0.1,0.448276,0.448276
1,0.2,0.520000,0.000000
2,0.3,0.545455,0.520000
3,0.4,0.578947,0.000000
4,0.5,0.600000,0.545455


## Part D

In [16]:
import altair as alt
alt.renderers.enable('notebook')

base = alt.Chart(df_roc, 
                 title='ROC Curve'
                ).properties(width=300)

roc_curve = base.mark_line(point=True).encode(
    alt.X('FPR', title='False Positive Rate (FPR)',  sort=None),
    alt.Y('TPR', title='True Positive Rate (TPR) (a.k.a Recall)'),
)

roc_rule = base.mark_line(color='green').encode(
    x='FPR',
    y='FPR',
    size=alt.value(2)
)

(roc_curve + roc_rule).interactive()

ValueError: 
To use the 'notebook' renderer, you must install the vega package
and the associated Jupyter extension.
See https://altair-viz.github.io/getting_started/installation.html
for more information.
